# Week 4
## Introduction
### Divide and Conquer Algorithm
**Divide**: Break into **non-overlapping** subproblems of the **same type**.

**Conquer**: solve subproblems and combine.

Because each subproblem is the same type as our original problem, it is naturally to solve the problems recursively. 

### Linear Search in Array
**Input**: An array $A$ with $n$ elements. A key $k$.

**Output**: An index, $i$, where $A[i]=k$. If there is no such $i$, then NOT_FOUND.

Below is the recursive version.

In [ ]:
LinearSearch(A, low, high, key):
    if high < low:
        return NOT_FOUND
    if A[low] == key:
        return low
    return LinearSearch(A, low+1, high, key)

A <span style="color:blue">recurrence relation</span> is an equation recursively defining a sequence of values. Example: Fibonacci numbers.

Recurrence defining worst-case time: $T(n) = T(n-1) + c$ and $T(0) = c$. Thus, the worse-case runtime for LinearSearch is $O(n)$.

The iterative version of LinearSearch is:

In [ ]:
LinearSearch(A, low, high, key):
    for i in range(low, high):
        if A[i] == key:
            return i
    return NOT_FOUND

### Binary Search
**Input**: A sorted array $A$, elements in $A$ can repeat, and a key $k$

**Output**: An index, $i$, where $A[i] = k$; Otherwise, the greatest index $i$, where $A[i] < k$; Otherwise, *low-1*.

In [ ]:
import math

def BinarySearch(A, low, high, key):
    if high < low:
        return low - 1
    mid = math.floor(low + (high - low)/2)
    if key = A[mid]:
        return mid
    else if key < A[mid]:
        return BinarySearch(A, low, mid - 1, key)
    else if key > A[mid]:
        return BinarySearch(A, mid + 1, high, key)

#### Binary Search Runtime Analysis

Binary search recurrence relation: $T(n) = T\left(\left\lfloor n/2 \right\rfloor\right) + c$, $T(0) = c$. There will be $log_2n$ times for an array of length $n$. So we could sum the time up and the total runtime is $O(log_2n)$. It can be written as $O(logn)$ as the base is not important.

**Iterative Version**

In [ ]:
import math

def BinarySearchIt(A, low, high, key):
    while low <= high:
        mid = math.floor(low + (high - low) / 2)
        if key = A[mid]:
            return mid
        else if key < A[mid]:
            high = mid - 1
        else:
            low = mid + 1
    return low - 1

## Polynomial Multiplication
Input: Two $n - 1$ degree polynomials: $A = (a_{n-1}, a_{n-2}, ..., a_1, a_0)$ and $B = (b_{n-1}, b_{n-2}, ..., b_1, b_0)$
\begin{align*}
&a_{n-1}x^{n-1} + a_{n-2}x^{n-2} + ... + a_1x + a_0 \\
&b_{n-1}x^{n-1} + b_{n-2}x^{n-2} + ... + b_1x + b_0
\end{align*}
Output: The product polynomial:
\begin{equation*}
c_{2n-2}x^{2n-2} + c_{2n-3}x^{2n-3} + ... + c_1x + c_0
\end{equation*}
where $c_{2n-2} = a_{n-1}b_{n-1}$, $c_{2n-3} = a_{n-1}b_{n-2} + a_{n-2}b_{n-1}$, ..., $c_1 = a_1b_0+a_0b_1$, $c_0=a_0b_0$.

Thus, we could see the naive algorithm is

In [5]:
import numpy as np

def MultPoly(A, B, n):
    product = np.zeros(2*n-1)
    for i in range(n-1):
        for j in range(n-1):
            product[i+j] += A[i]*B[j]
    return product

The runtime for `MultPoly()` function is $O(n^2)$.

### Naive Divide and Conquer Algorithm
Let $A(x) = D_1(x)x^{\frac{n}{2}} + D_0(x)$ where
\begin{equation*}
    D_1(x) = a_{n-1}x^{\frac{n}{2}-1} + a_{n-2}x^{\frac{n}{2}-2} + ... + a_{\frac{n}{2}} \\
    D_0(x) = a_{\frac{n}{2}-1}x^{\frac{n}{2}-1} + a_{\frac{n}{2}-2}x^{\frac{n}{2}-2} + ... + a_0
\end{equation*}
and $B(x) = E_1(x)x^{\frac{n}{2}} + E_0(x)$ where $E_1(x)$ and $E_0(x)$ have similar form as $D(x)$.

Then, we have 
\begin{align*}
    AB &= (D_1x^{\frac{n}{2}} + D_0)(E_1x^{\frac{n}{2}} + E_0) \\
       &= (D_1E_1)x^n + (D_1E_0 + D_0E_1)x^{\frac{n}{2}} + D_0E_0
\end{align*}

Recurrence: the runtime is $T(n)=4T(\frac{n}{2})+kn$.

In [ ]:
import numpy as np

def Mult2(A, B, n, ai, bi):
    """
    ai, bi: coefficients that we are interested in.
    """
    R = np.zeros(2*n-1)
    if n = 1:
        R[0] = A[ai] * B[bi]
        return R
    R[0:n-2] = Mult2(A, B, n/2, ai, bi)
    R[n:2n-2] = Mult2(A, B, n/2, ai+n/2, bi+n/2)
    D0E1 = Mult2(A, B, n/2, ai, bi+n/2)
    D1E0 = Mult2(A, B, n/2, ai+n/2, bi)
    R[n/2:n+n/2-2] += D1E0 + D0E1
    return R

`Mult2()` function requires that $n$ is a power of 2 so the code can safely calculate $\frac{n}{2}$ without worrying about rounding. It can be achieved by padding each polynomial with zero terms to the needed degree.

The runtime is $\sum_{i=0}^{log_2n}4^ik\frac{n}{2^i} = O(n^2)$, which is the same as the naive algorithm.

### Karatsuba approach
Rewrite $C(x) = a_1b_1x^2 + (a_1b_0 + a_0b_1)x + a_0b_0$ as 
\begin{equation*}
    C(x) = a_1b_1x^2 + ((a_1 + a_0)(b_1 + b_0) - a_1b_1 - a_0b_0)x + a_0b_0
\end{equation*}
which only needs 3 multiplications.

The runtime is $\sum_{i=0}^{log_2n}k\frac{n}{2^i}=O(n^{log_23})=O(n^{1.58})$. Note: the extra additions change the $k$ constant here.